# Saxena Carbon Database 
Required system packages and initialization

In [ ]:
import pandas as pd
import numpy as np
import sympy as sym
import os
import thermoengine

import pyximport
import importlib
sym.init_printing()


import matplotlib.pyplot  as plt
from fuzzywuzzy import fuzz
# python-Levenshtein


In [ ]:
def load_coder_modules(working_dir='working'):
    pyximport.install(language_level=3)
    
    working_dir_code = np.array(os.listdir(working_dir))
    mask = np.array([file.endswith('.pyx') for file in working_dir_code ])
    mod_names = [os.path.splitext(file)[0] for file in working_dir_code[mask]]
    
    os.chdir('working')
    [importlib.import_module(mod_name) for mod_name in mod_names]
    os.chdir('..')

In [ ]:
def lookup_phase_info(phase_name, match_thresh=80):
    phase_info = thermoengine.phases.get_phase_info()[0]
    pure_info = phase_info['pure']
    soln_info = phase_info['solution']
    
    scores = []
    for iname in pure_info['Name']:
        scores.append(fuzz.token_set_ratio(phase_name, iname))
        
    scores = np.array(scores)
    
    inds = np.argsort(scores)[::-1]
    mask = scores[inds]>match_thresh
    inds = inds[mask]
    
    matches = pure_info.iloc[inds]
    return matches
    

In [ ]:
def make_custom_database(mod_name, phase_params, 
                         working_dir='working'):
    
    modelDB = None
    coder_mod = None
    
    model.set_module_name(mod_name)
    !mkdir -p {working_dir}
    %cd {working_dir}
    
    try:
        phase_info = {}
        
        sources_list = ''
        os.system('rm '+mod_name+'.pyx')
        
        for phs in phase_params:
            param_dict = phase_params[phs]
            phs_nm = param_dict['Phase']
            phase_info[phs] = [phs_nm, 'pure']
            
            sources_list = _add_model_phase(
                param_dict, mod_name, sources_list=sources_list)
            
        pyxbld_filenm = mod_name+'.pyxbld'
        _rewrite_pyxbld_sources(pyxbld_filenm, sources_list)
        
        
        phase_tuple =  (mod_name, phase_info)
        
        coder_mod = importlib.import_module(mod_name)
        
        
        modelDB = thermoengine.model.Database(
            database='CoderModule', phase_tuple=phase_tuple)
    except:
        print('There is something wrong with this custom database attempt')
        raise
    
    %cd ..
    return modelDB, coder_mod


In [ ]:

def _add_model_phase(param_dict, mod_name, sources_list=None, 
                    model_type='calib'):
    filenm = mod_name+'.pyx'
    tmpnm = 'tmpfile.pyx'
    tmpnm2 = 'tmpfile2.pyx'
    
    os.system('touch '+ filenm)
    os.system('mv '+filenm+' '+tmpnm)
    
    
    phase_name = param_dict.pop('Phase', None).title()
    formula = param_dict.pop('Formula', None)
    result = model.create_code_module(phase=phase_name, formula=formula, 
                                      params=param_dict, module_type=model_type)
    
    os.system('cat '+filenm+' '+tmpnm+' > '+tmpnm2)
    os.system('mv '+tmpnm2 + ' ' + filenm)
    os.system('rm '+tmpnm)
    os.system('rm '+tmpnm2)
    
    
    if sources_list is None:
        sources_list = ''
        
    
    source_file = phase_name+'_'+mod_name+'_calib.c'
    sources_list += "'"+source_file + "', "
    
    return sources_list


def _rewrite_pyxbld_sources(filename, sources_list):
    with open(filename,'r') as file:
        lines = file.readlines()
        for ind, line in enumerate(lines):
            is_source_line = line.strip().startswith('sources')
            if is_source_line:
                ind_source = ind
            
        file.close()
        
        
    sources_update = ('                     sources=[pyxfilename, '+
                      sources_list[:-2] + '],\r\n')
    
    lines[ind_source] = sources_update
    
    with open(filename,'w') as file:
        file.writelines( lines )
        